In [ ]:
!pip install azure-ai-inference[opentelemetry]
!pip install azure-search-documents 
!pip install azure-identity
!pip install openai

## Setup API Clients

In [1]:
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
from azure.search.documents import SearchClient
import dotenv
from azure.search.documents.models import VectorizedQuery, VectorizableTextQuery

dotenv.load_dotenv()

# Load Azure OpenAI environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")

# Load Azure Search environment variables
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")

# 🔹 Initialize Azure OpenAI Client (API Key or Managed Identity)
if AZURE_OPENAI_API_KEY:
    openai_client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_version="2024-10-21"
    )
else:
    azure_credential = DefaultAzureCredential()
    token_provider = get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
    openai_client = AzureOpenAI(
        azure_ad_token_provider=token_provider,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_version="2024-10-21"
    )

# 🔹 Initialize Azure AI Search Client (API Key or Managed Identity)
if AZURE_SEARCH_ADMIN_KEY:
    search_client = SearchClient(
        endpoint=AZURE_SEARCH_ENDPOINT,
        index_name=AZURE_SEARCH_INDEX_NAME,
        credential=AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
    )
else:
    azure_credential = DefaultAzureCredential()
    search_client = SearchClient(
        endpoint=AZURE_SEARCH_ENDPOINT,
        index_name=AZURE_SEARCH_INDEX_NAME,
        credential=azure_credential
    )

def get_embedding(text):
    return openai_client.embeddings.create(
        model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME"),
        input=text
    ).data[0].embedding

## Prepare a question

In [2]:
user_question = "What is included in my Northwind Health Plus plan that is not in standard?"
user_question_vector = get_embedding(user_question)
print(user_question_vector)

[-0.008389439433813095, 0.0077372328378260136, 0.04956771805882454, 0.048194654285907745, -0.045201364904642105, -0.0022621280513703823, -0.0579434297978878, 0.003810260910540819, 0.0030550742521882057, 0.016820071265101433, 0.021337462589144707, 0.011156169697642326, 0.022229954600334167, 0.05198431760072708, -0.024536708369851112, 0.03147069737315178, -0.02124134637415409, 0.015941308811306953, -0.008519880473613739, 0.048798803240060806, 0.004445304628461599, 0.04602520540356636, 0.011890760622918606, -0.029493479058146477, -0.0632709264755249, -0.02982301637530327, -0.04023086279630661, -0.0033880427945405245, 0.03992879018187523, -0.005615844391286373, 0.015350889414548874, -0.03583705052733421, 0.01127974595874548, 0.006697134580463171, -0.014568241313099861, -0.026376618072390556, 0.039873868227005005, 0.03103131428360939, 0.031525619328022, 0.0290815606713295, -0.022312339395284653, 0.01753406599164009, -0.009172087535262108, 0.0301250908523798, 0.028422487899661064, 0.00889060

## Retrieve matching documents

In [3]:
search_results = search_client.search(
    None,
    top=10,
    vector_queries=[
        VectorizableTextQuery( # Using Azure AI Search vectorizer
            text=user_question, k_nearest_neighbors=10, fields="text_vector"
        )
    ],
)

# Print Results
for result in search_results:
    print("Chunk ID:", result["chunk_id"])
    print("Title:", result["title"])
    print("Text:", result["chunk"])
    print()

Chunk ID: 10331bdf9a20_aHR0cHM6Ly9zdGZzdW5hdmFsYXNhbmRib3hkZXZldXMuYmxvYi5jb3JlLndpbmRvd3MubmV0L2NvbnRvc28taHIvQmVuZWZpdF9PcHRpb25zLnBkZg2_pages_1
Title: Benefit_Options.pdf
Text: a variety of in-network providers, including primary care 
physicians, specialists, hospitals, and pharmacies. This plan does not offer coverage for emergency 
services, mental health and substance abuse coverage, or out-of-network services.

Comparison of Plans 
Both plans offer coverage for routine physicals, well-child visits, immunizations, and other preventive 
care services. The plans also cover preventive care services such as mammograms, colonoscopies, and 
other cancer screenings. 

Northwind Health Plus offers more comprehensive coverage than Northwind Standard. This plan offers 
coverage for emergency services, both in-network and out-of-network, as well as mental health and 
substance abuse coverage. Northwind Standard does not offer coverage for emergency services, mental 
health and substance ab

## RAG TIME!

In [5]:
# First, let's collect the context from search results
context = ""
for result in search_results:
    context += result["chunk"] + "\n\n"

SYSTEM_MESSAGE = f"""
You are an AI Assistant.
Be brief in your answers. Answer ONLY with the facts listed in the retrieved text.

Context:
{context}
"""

USER_MESSAGE = user_question

response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME"),
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)


The Northwind Health Plus plan includes additional benefits such as preventive care, wellness programs, and enhanced coverage options that are not typically included in standard plans.
